## Rechtspraak ETL

In [1]:
# # # EXTRA MEMORY
# d=[]
# while(1):
#   d.append('1')

In [ ]:
#!pip install mysql.connector

In [ ]:
# # Install RAPIDS
# !git clone https://github.com/rapidsai/rapidsai-csp-utils.git
# !bash rapidsai-csp-utils/colab/rapids-colab.sh stable

In [ ]:
# import sys, os

# dist_package_index = sys.path.index('/usr/local/lib/python3.6/dist-packages')
# sys.path = sys.path[:dist_package_index] + ['/usr/local/lib/python3.6/site-packages'] + sys.path[dist_package_index:]
# sys.path
# exec(open('rapidsai-csp-utils/colab/update_modules.py').read(), globals())

In [ ]:
#!nvidia-smi

In [ ]:
#!apt install unixodbc-dev
#!pip install pyodbc

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from sqlalchemy import create_engine

In [ ]:
SERVER = '145.100.58.122'
DATABASE = 'case_law'
PORT = '3306'
USERNAME = 'root'
PASSWORD = 'weboflaw2020'
CONNECTION = f'mysql+mysqlconnector://{USERNAME}:{PASSWORD}@{SERVER}:{PORT}/{DATABASE}'

In [ ]:
engine = create_engine(CONNECTION, echo=True)
connection = engine.connect()

In [ ]:
query = """SELECT * FROM case_law.case LIMIT 1;"""
check = pd.read_sql_query(query, con=connection)
check.head()

### MYSQL

In [ ]:
import mysql.connector as mdb
from mysql.connector import Error
import sys, traceback

In [ ]:
def open_database(host, db_name, username, password):
    try:
        connection = mdb.connect(host=host,
                                 database=db_name,
                                 user=username,
                                 password=password)
        if connection.is_connected():
           db_Info = connection.get_server_info()
           print("Connected to MySQL database... MySQL Server version on ",db_Info)
           cursor = connection.cursor()
           cursor.execute("select database();")
           record = cursor.fetchone()
           print ("Your connected to - ", record)
           cursor.close()
    except Error as e :
        print ("Error while connecting to MySQL", e)
    return connection

In [ ]:
def close_database(connection):
    #closing database connection.
    if(connection.is_connected()):
        connection.close()
        print("MySQL connection is closed")

In [ ]:
conn = open_database('145.100.58.122', 'case_law', 'root', 'weboflaw2020')

In [ ]:
# query = """SELECT * FROM case_law.case LIMIT 1;"""
# cursor = conn.cursor(buffered=True)
# cursor.execute(query)
# records = cursor.fetchall()
# for row in records:
#     print(row)

---
### Datasets

In [ ]:
# import cudf as pd
# import io, requests

In [ ]:
df_case_raw = pd.read_csv("https://maastrichtuniversity-ids-open.s3.eu-central-1.amazonaws.com/rechtspraak/case.csv").replace(np.nan, 'NULL', regex=True)

In [ ]:
#df_case_raw.head(2)

In [ ]:
df_country = pd.read_csv("https://maastrichtuniversity-ids-open.s3.eu-central-1.amazonaws.com/rechtspraak/countries.csv").replace(np.nan, 'NULL', regex=True)

In [ ]:
#df_country.head(2)

In [ ]:
df_li_cases = pd.read_csv("https://maastrichtuniversity-ids-open.s3.eu-central-1.amazonaws.com/rechtspraak/legal_intelligence_cases.csv").replace(np.nan, 'NULL', regex=True).drop_duplicates(subset ="ecli")

In [ ]:
#df_li_cases.head(2)

In [ ]:
df_case_opinion = pd.read_csv("https://maastrichtuniversity-ids-open.s3.eu-central-1.amazonaws.com/rechtspraak/case_opinion_from_advocate_general.csv").replace(np.nan, 'NULL', regex=True)

In [ ]:
#df_li_cases.head(2)

In [ ]:
df_case_citation = pd.read_csv("https://maastrichtuniversity-ids-open.s3.eu-central-1.amazonaws.com/rechtspraak/caselaw_citations.csv").replace(np.nan, 'NULL', regex=True)

In [ ]:
#df_case_citation.head(2)

In [ ]:
df_legislation_citation = pd.read_csv("https://maastrichtuniversity-ids-open.s3.eu-central-1.amazonaws.com/rechtspraak/legislation_citations.csv").replace(np.nan, 'NULL', regex=True)

In [ ]:
#df_legislation_citation.head(2)

---
### Normalization based on constrains

In [ ]:
#li formating
for i, li_case in df_li_cases.iterrows():
    ecli = li_case.ecli
    new_ecli = ecli.replace('_', ':')
    df_li_cases.at[i, 'ecli'] = new_ecli

li_intersection = list(set(df_li_cases['ecli']).intersection(set(df_case_raw['case_id'])))

#df_case 
sample_size = 5000
uniques = list(df_case_citation['source_ecli'].sample(n=sample_size, random_state=18))
uniques.extend(li_intersection)

df_case = df_case_raw[df_case_raw['case_id'].isin(uniques)]

##1: legislation citations 
df_legislation_citation = df_legislation_citation[df_legislation_citation['source_ecli'].isin(uniques)].reset_index(drop=True)

##2: case citations 
df_case_citation = df_case_citation[df_case_citation['source_ecli'].isin(uniques)].reset_index(drop=True)

##3: case opinions 
df_case_opinion = df_case_opinion[df_case_opinion['case_id'].isin(uniques)].reset_index(drop=True)

##4: li cases 
df_li_cases = df_li_cases[df_li_cases['ecli'].isin(uniques)].reset_index(drop=True)

print('Number of Cases in\nCases: {}\nLegislation citation: {}\nCase citation: {}\nCase opinion: {}\nLI cases: {}'\
      .format(len(df_case), len(df_legislation_citation), len(df_case_citation), len(df_case_opinion), len(df_li_cases)))

In [ ]:
del uniques

---
### Utils

In [ ]:
def to_tuples(df):
    return [tuple(x) for x in df.values]

In [ ]:
import string

def clean_strings(column):
  cleaned = []
  for s in column:
    cleaned.append("".join(filter(lambda char: char in string.printable, s)))
  return [i[0:250] for i in cleaned]

In [ ]:
def clean_table_sql(table_name):
    try:
        cursor1 = conn.cursor(buffered=True)
        query1 = """DELETE FROM `{}`;""".format(table_name)
        cursor1.execute(query1)
        cursor2 = conn.cursor(buffered=True)
        query2 = """ALTER TABLE `{}` AUTO_INCREMENT = 1;""".format(table_name)
        cursor2.execute(query2)
        conn.commit()
    except Error as error :
        conn.rollback()
        print("Failed to delete MySQL table {}".format(error))
        traceback.print_exc(file=sys.stdout)

In [ ]:
def clean_table_sql(table_name):
    try:
        cursor1 = conn.cursor(buffered=True)
        query1 = """DELETE FROM `{}`;""".format(table_name)
        cursor1.execute(query1)
        cursor2 = conn.cursor(buffered=True)
        query2 = """ALTER TABLE `{}` AUTO_INCREMENT = 1;""".format(table_name)
        cursor2.execute(query2)
        conn.commit()
    except Error as error :
        conn.rollback()
        print("Failed to delete MySQL table {}".format(error))
        traceback.print_exc(file=sys.stdout)

In [ ]:
c.execute('''  
SELECT Brand, max(price) FROM CARS
          ''')

In [ ]:
def get_parent_ids(table, column_table, df, column_df):
  """DB table, DB column_table, df: pandas df to look at, column_df"""
  query =  f"""SELECT id, {column_table} FROM {table}"""
  read_all_ids = pd.read_sql(query, con=connection)
  id_list = []
  for idx, data in enumerate(df[column_df]):
    id_list.append(read_all_ids[read_all_ids[column_table] == data].id.values[0])
  return id_list

---
### Tables Hierarchies

1. Court
2. Case
3. Case Opinion
4. LI Case
5. Subject
6. Case subject
7. Country
8. Case country
9. Case citation
10. Legislation citation
11. Case related decision


In [ ]:
clean_table_sql('case_related_decision')
clean_table_sql('legislation_citation')
clean_table_sql('case_citation')
clean_table_sql('case_country')
clean_table_sql('country')
clean_table_sql('case_subject')
clean_table_sql('subject')
clean_table_sql('legal_intelligence_case')
clean_table_sql('case_opinion')
clean_table_sql('case')
clean_table_sql('court')

---
### Courts

In [ ]:
court = pd.DataFrame()

In [ ]:
courts_list = df_case_raw.authority.unique()
courts_list = [i.replace('"','-') for i in courts_list]
court['name'] = clean_strings(courts_list)

In [ ]:
#dummies
court.loc[:,'type'] = 'NULL'
court.loc[:,'level'] = 'NULL'
court.loc[:,'country'] = 'NULL'
court.loc[:,'language'] = 'NULL' 
court.loc[:,'jurisdiction'] = 'NULL' 
court.loc[:,'law_area'] = 'NULL'
court.loc[:,'authority_level'] = 'NULL'

In [ ]:
court.head(2)

In [ ]:
court.to_sql('court', con=engine, index=False, if_exists='append', chunksize = 10)

In [ ]:
len(court)

In [ ]:
del court
#del tuples

---
### Case

In [ ]:
map_case = {'date':'date',
            'description':'description',
            'language':'language',
            'venue':'venue',
            'abstract':'abstract',
            'procedure_type':'procedure_type',
            'lodge_date':'lodge_date',
            'alternative_sources':'link',
            'case_id':'ecli'}

In [ ]:
case = df_case[map_case.keys()].rename(columns=map_case)

In [ ]:
parents_ids = get_parent_ids('court', 'name', df_case, 'authority')

In [ ]:
case['name'] = 'NULL'
case['court_id'] = parents_ids
case['date'] = [pd.to_datetime(i, errors='coerce') if i != 'NULL' else pd.to_datetime('1900-01-01 00:00:00') for i in case['date']]
case['lodge_date'] = [pd.to_datetime(i, errors='coerce') if i != 'NULL' else pd.to_datetime('1900-01-01 00:00:00') for i in case['lodge_date']]
case['description'] = clean_strings(case['description'])
case['link'] = clean_strings(case['link'])

In [ ]:
case.head(3)

In [ ]:
case.to_sql('case', con=engine, index=False, if_exists='append', chunksize = 100)

In [ ]:
del map_case
del case

---
### Case opinion advocate general

In [ ]:
map_case_opinion = {'date':'date',
                    'case_number':'case_number',
                    'description':'description',
                    'language':'language',
                    'country':'country',
                    'venue':'venue',
                    'abstract':'abstract',
                    'procedure_type':'procedure_type',
                    'authority':'authority',
                    'case_id':'ecli'}

In [ ]:
case_opinion = df_case_opinion[map_case_opinion.keys()].rename(columns=map_case_opinion)

In [ ]:
case_opinion['date'] = [pd.to_datetime(i, errors='coerce') if i != 'NULL' else pd.to_datetime('1900-01-01 00:00:00') for i in case_opinion['date']]
case_opinion['abstract'] = clean_strings(case_opinion['abstract'])

In [ ]:
case_opinion.head(2)

In [ ]:
case_opinion.to_sql('case_opinion', con=engine, index=False, if_exists='append', chunksize = 100)

In [ ]:
del map_case_opinion
del case_opinion

---
### Legal Intelligence Cases

In [ ]:
map_legal_intelligence_case = {
    'ecli':'ecli',
    'Title':'name',
    'date':'date',
    'abstract':'abstract',
    'LawArea':'subject',
    'Url':'link',
    'DisplayTitle':'DisplayTitle',
    'OriginalUrl':'OriginalUrl',
    'Jurisdiction':'Jurisdiction',
    'DocumentType':'DocumentType',
    'case_number':'CaseNumber',
    'PublicationNumber':'PublicationNumber',
    'IssueNumber':'IssueNumber',
    'lodge_date':'lodge_date',
    'DateAdded':'DateAdded',
    'Sources':'Sources',
    'UrlWithAutoLogOnToken':'UrlWithAutoLogOnToken',
    'authority':'court',
    'DisplaySubtitle':'DisplaySubtitle'
}

In [ ]:
legal_intelligence_case = df_li_cases[map_legal_intelligence_case.keys()].rename(columns=map_legal_intelligence_case)

In [ ]:
legal_intelligence_case.head(2)

In [ ]:
legal_intelligence_case['date'] = [pd.to_datetime(i, errors='coerce') if i != 'NULL' else pd.to_datetime('1900-01-01 00:00:00') for i in df_li_cases['date']]
legal_intelligence_case['lodge_date'] = [pd.to_datetime(i, errors='coerce') if i != 'NULL' else pd.to_datetime('1900-01-01 00:00:00') for i in df_li_cases['lodge_date']]
legal_intelligence_case['DateAdded'] = [pd.to_datetime(i, errors='coerce') if i != 'NULL' else pd.to_datetime('1900-01-01 00:00:00') for i in df_li_cases['DateAdded']]
legal_intelligence_case['name'] = clean_strings(legal_intelligence_case['name'])
legal_intelligence_case['abstract'] = clean_strings(legal_intelligence_case['abstract'])
legal_intelligence_case['DisplayTitle'] = clean_strings(legal_intelligence_case['DisplayTitle'])
legal_intelligence_case['court'] = clean_strings(legal_intelligence_case['court'])
legal_intelligence_case['DisplaySubtitle'] = clean_strings(legal_intelligence_case['DisplaySubtitle'])
#legal_intelligence_case['name'] = [i[0:250] for i in legal_intelligence_case['name']]
#legal_intelligence_case['DisplayTitle'] = [i[0:250] for i in legal_intelligence_case['DisplayTitle']]
#legal_intelligence_case['DisplaySubtitle'] = [i[0:250] for i in legal_intelligence_case['DisplaySubtitle']]

In [ ]:
legal_intelligence_case.to_sql('legal_intelligence_case', con=engine, index=False, if_exists='append', chunksize = 10)

In [ ]:
del map_legal_intelligence_case
del legal_intelligence_case

---
### Subjects

In [ ]:
subjects_as_list = [list(row.split("; ")) for row in df_case.subject]
unique_subjects = \
    set(list(
        pd.core.common\
            .flatten(subjects_as_list)))

In [ ]:
subject = pd.DataFrame()
subject['name'] = clean_strings(list(sorted(unique_subjects)))
subject.loc[:,'standard_name'] = 'NULL'

In [ ]:
subject.head(2)

In [ ]:
subject.to_sql('subject', con=engine, index=False, if_exists='append', chunksize = 50)

In [ ]:
del subject

---
### Case - Subject

In [ ]:
df_subjects_case = df_case[['subject','case_id']]
df_subjects_case.loc[:,'subject'] = subjects_as_list
df_subjects_case = df_subjects_case.explode('subject')
len(df_subjects_case)

In [ ]:
parents_ids_subjects = get_parent_ids('subject', 'name', df_subjects_case, 'subject')
len(parents_ids_subjects)

In [ ]:
parents_ids_cases = get_parent_ids('case', 'ecli', df_subjects_case, 'case_id')
len(parents_ids_cases)

In [ ]:
case_subject = pd.DataFrame({'case_id':parents_ids_cases,
                             'subject_id':parents_ids_subjects})

In [ ]:
#case_subject.sort_values(by='case_id').tail(50) #quality check
#[(type(i), type(j)) for i,j in zip(case_subject.subject_id,case_subject.case_id)]

In [ ]:
case_subject.tail(3)

In [ ]:
case_subject.to_sql('case_subject', con=engine, index=False, if_exists='append', chunksize = 100)

In [ ]:
# tuples = to_tuples(case_subject)
# tuples = [(int(i), int(j)) for i,j in tuples]

In [ ]:
# try:  
#     cursor = conn.cursor()
#     for idx, data in enumerate(tuples):
#         query = ("INSERT INTO `case_subject`"
#                 "(case_id, subject_id)"
#                 "VALUES (%s, %s)")
#         cursor.execute(query, data)
#     conn.commit()
#     cursor.close()
#     print('case_subject added')
# except Error as error :
#     conn.rollback()
#     print("Failed to insert into MySQL table {}".format(error))
#     traceback.print_exc(file=sys.stdout)

In [ ]:
del df_subjects_case
del case_subject
#del tuples

---
### Countries

In [ ]:
df_country.head()

In [ ]:
df_country.loc[:,'language'] = 'NULL'
df_country.loc[:,'eea'] = 0

In [ ]:
country = df_country.rename(columns={'country_id':'id'})

In [ ]:
country.head(3)

In [ ]:
country.to_sql('country', con=engine, index=False, if_exists='append', chunksize = 100)

In [ ]:
#tuples = to_tuples(country)

In [ ]:
# try:  
#     cursor = conn.cursor()
#     for idx, data in enumerate(tuples):
#         query = ("INSERT INTO `country`"
#                 "(id, name, language, flag, eu, eea)"
#                 "VALUES (%s, %s, %s, %s, %s, %s)")
#         cursor.execute(query, data)
#     conn.commit()
#     cursor.close()
#     print('country added')
# except Error as error :
#     conn.rollback()
#     print("Failed to insert into MySQL table {}".format(error))
#     traceback.print_exc(file=sys.stdout)

In [ ]:
del country
#del tuples

---
### Case - Country

In [ ]:
df_case.head(2)

In [ ]:
#as_list = [list(row.split("; ")) for row in df_case.country] #in case there is more than one country in the row

In [ ]:
df_country_case = df_case[['case_id','country']]
#df_country_case.loc[:,'country'] = as_list
df_country_case = df_country_case.explode('country')
len(df_country_case)

In [ ]:
parents_ids_countries = get_parent_ids('country', 'id', df_country_case, 'country')
len(parents_ids_countries)

In [ ]:
parents_ids_cases = get_parent_ids('case', 'ecli', df_country_case, 'case_id')
len(parents_ids_cases)

In [ ]:
case_country = pd.DataFrame({'case_id':parents_ids_cases,
                             'country_id':parents_ids_countries})

In [ ]:
case_country.tail(3)

In [ ]:
case_country.to_sql('case_country', con=engine, index=False, if_exists='append', chunksize = 100)

In [ ]:
#tuples = to_tuples(case_country)
#tuples = [(int(i), int(j)) for i,j in tuples]

In [ ]:
# try:  
#     cursor = conn.cursor()
#     for idx, data in enumerate(tuples):
#         query = ("INSERT INTO `case_country`"
#                 "(case_id, country_id)"
#                 "VALUES (%s, %s)")
#         cursor.execute(query, data)
#     conn.commit()
#     cursor.close()
#     print('case_subject added')
# except Error as error :
#     conn.rollback()
#     print("Failed to insert into MySQL table {}".format(error))
#     traceback.print_exc(file=sys.stdout)

In [ ]:
del df_country_case
del case_country
#del tuples

---
### Case law citation

In [ ]:
df_case_citation.head(4)

In [ ]:
parent_ids = get_parent_ids('case', 'ecli', df_case_citation, 'source_ecli')
df_case_citation['case_id'] = parent_ids

In [ ]:
map_case_citation = {
    'source_ecli':'source_ecli',
    'source_paragraph':'source_paragraph',
    'target_ecli':'target_ecli',
    'target_paragraph':'target_paragraph',
    'case_id':'case_id'
}

In [ ]:
case_citation = df_case_citation[map_case_citation.keys()].rename(columns=map_case_citation)

In [ ]:
case_citation.to_sql('case_citation', con=engine, index=False, if_exists='append', chunksize = 100)

In [ ]:
#tuples = to_tuples(case_citation)

In [ ]:
# try:  
#     cursor = conn.cursor()
#     for idx, data in enumerate(tuples):
#         query = ("INSERT INTO `case_citation`"
#                 "(source_ecli, source_paragraph, target_ecli, target_paragraph, case_id)"
#                 "VALUES (%s, %s, %s, %s, %s)")
#         cursor.execute(query, data)
#     conn.commit()
#     cursor.close()
#     print('case_citation added')
# except Error as error :
#     conn.rollback()
#     print("Failed to insert into MySQL table {}".format(error))
#     traceback.print_exc(file=sys.stdout)

In [ ]:
del case_citation
#del tuples

---
### Legislation citation

In [ ]:
df_legislation_citation.head(4)

In [ ]:
parent_ids = get_parent_ids('case', 'ecli', df_legislation_citation, 'source_ecli')
df_legislation_citation['case_id'] = parent_ids

In [ ]:
df_legislation_citation.loc[:,'target_name'] = 'NULL'
df_legislation_citation.loc[:,'target_sourcename'] = 'NULL'

In [ ]:
map_legislation_citation = {
    'source_ecli':'source_ecli',
    'source_paragraph':'source_paragraph',
    'target_id':'target_article',
    'target_paragraph':'target_article_paragraph',
    'target_name':'target_name',
    'target_sourcename':'target_sourcename',
    'target_link':'target_article_webpage',
    'case_id':'case_id'
}

In [ ]:
legislation_citation = df_legislation_citation[map_legislation_citation.keys()].rename(columns=map_legislation_citation)

In [ ]:
legislation_citation.to_sql('legislation_citation', con=engine, index=False, if_exists='append', chunksize = 100)

In [ ]:
#tuples = to_tuples(legislation_citation)

In [ ]:
# try:  
#     cursor = conn.cursor()
#     for idx, data in enumerate(tuples):
#         query = ("INSERT INTO `legislation_citation`"
#                 "(source_ecli, source_paragraph, target_id, target_paragraph, target_name, target_sourcename, target_link, case_id)"
#                 "VALUES (%s, %s, %s, %s, %s, %s, %s, %s)")
#         cursor.execute(query, data)
#     conn.commit()
#     cursor.close()
#     print('legislation_citation added')
# except Error as error :
#     conn.rollback()
#     print("Failed to insert into MySQL table {}".format(error))
#     traceback.print_exc(file=sys.stdout)

In [ ]:
del legislation_citation
#del tuples

---
### Case related decision

In [ ]:
df_case_related = df_case[['case_id','related_cases']]\
    .rename(columns = {'case_id':'source_ecli', 
                       'related_cases': 'referencing_case_ecli'})

In [ ]:
parent_ids = get_parent_ids('case', 'ecli', df_case_related, 'source_ecli')
df_case_related['case_id'] = parent_ids

In [ ]:
df_case_related.loc[:,'referenced_case_ecli'] = 'NULL'

In [ ]:
df_case_related.head(2)

In [ ]:
map_case_related_decision = {
    'source_ecli':'source_ecli',
    'referencing_case_ecli':'referencing_case_ecli',
    'referenced_case_ecli':'referenced_case_ecli',
    'case_id':'case_id'
    }

In [ ]:
case_related_decision = df_case_related[map_case_related_decision.keys()].rename(columns=map_case_related_decision)

In [ ]:
case_related_decision.head(3)

In [ ]:
case_related_decision.to_sql('case_related_decision', con=engine, index=False, if_exists='append', chunksize = 100)

In [ ]:
#tuples = to_tuples(case_related_decision)

In [ ]:
# try:  
#     cursor = conn.cursor()
#     for idx, data in enumerate(tuples):
#         query = ("INSERT INTO `case_related_decision`"
#                 "(source_ecli, referencing_case_ecli, referenced_case_ecli, case_id)"
#                 "VALUES (%s, %s, %s, %s)")
#         cursor.execute(query, data)
#     conn.commit()
#     cursor.close()
#     print('case_related_decision added')
# except Error as error :
#     conn.rollback()
#     print("Failed to insert into MySQL table {}".format(error))
#     traceback.print_exc(file=sys.stdout)

In [ ]:
del case_related_decision
#del tuples